# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from fastcore.foundation import L
from fastcore.utils import *

In [ ]:
#| export
def foo(bogus_param): 
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()